In [1]:
# Uninstall potentially conflicting pre-installed versions (optional, but ensures a clean state)
!pip uninstall -qqy protobuf six wheel typing-extensions

# Install a compatible version of protobuf
# Version 4.23.4 or 3.20.x are often good choices depending on the TF version used in Kaggle
!pip install protobuf==4.23.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 7.7 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
model-signing 1.1.1 requires typing-extensions, which is not installed.
onnx 1.18.0 requires typing_extensions>=4.7.1, which is not installed.
google-cloud-aiplatform 1.125.0 requires typing_extensions, which is not installed.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires typing-extensions>=4.5.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
bigframes 2.12.0 requires typing-extensions<5,>=4.5.0, which is not installed.
google-generativeai 0.8.5 requires typing-extensions, which is not installed.
wandb 0.21.0 requires typing-extensions<5,>=4.8, which is not installed.
tensorflow-decision-forests 1.11.0 requires wheel, which is not installed.
tensor

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import load_img, img_to_array

2025-11-17 14:30:13.653733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763389813.874225      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763389813.927945      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
IMG_SIZE = (224, 224)  # A good standard size
BATCH_SIZE = 32
SEED = 42
NUM_CLASSES = 10
train_dir = "/kaggle/input/new-paddy-doctor-paddy-disease-classification/paddy-disease-classification/train_images"
test_dir = "/kaggle/input/new-paddy-doctor-paddy-disease-classification/paddy-disease-classification/test_images"

In [4]:
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
class_names = train_ds.class_names

Found 19131 files belonging to 10 classes.


I0000 00:00:1763389841.163733      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763389841.164448      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 10407 files belonging to 10 classes.


In [5]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
    # --- ADD THESE LINES ---
    layers.RandomContrast(0.1),  # Randomly adjust contrast by up to 10%
    layers.RandomBrightness(0.1), # Randomly adjust brightness by up to 10%
    # -----------------------
  ],
  name="data_augmentation"
)

In [6]:
# Apply augmentation to the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                        num_parallel_calls=tf.data.AUTOTUNE)

# Apply the EfficientNet-specific preprocessing to BOTH datasets
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y),
                        num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y),
                      num_parallel_calls=tf.data.AUTOTUNE)

# Configure datasets for performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
base_model = EfficientNetV2S(
    include_top=False, 
    weights='imagenet',   
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

base_model.trainable = False


inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

x = base_model(inputs, training=False)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x) 
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x) # Final 10-class output


model_transfer_learning = Model(inputs, outputs)


model_transfer_learning.summary()

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-s (Functional)   │ (None, 7, 7, 1280)     │    20,331,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,344,170 (77.61 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 20,331,360 (77.56 MB)

In [8]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=1,
    min_lr=0.000001
)


In [9]:
model_transfer_learning.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy', # Use this because your labels are integers
    metrics=['accuracy']
)

In [10]:
history = model_transfer_learning.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[lr_scheduler]
)

Epoch 1/15


I0000 00:00:1763389875.674703     128 service.cc:148] XLA service 0x7b5c600028c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763389875.675606     128 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763389875.675624     128 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763389879.811252     128 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/598 ━━━━━━━━━━━━━━━━━━━━ 7:48:17 47s/step - accuracy: 0.0625 - loss: 2.4127

I0000 00:00:1763389900.247506     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


598/598 ━━━━━━━━━━━━━━━━━━━━ 255s 348ms/step - accuracy: 0.2702 - loss: 2.0343 - val_accuracy: 0.4274 - val_loss: 1.5904 - learning_rate: 0.0010
Epoch 2/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 163s 272ms/step - accuracy: 0.4272 - loss: 1.6561 - val_accuracy: 0.4550 - val_loss: 1.5434 - learning_rate: 0.0010
Epoch 3/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 163s 271ms/step - accuracy: 0.4551 - loss: 1.5699 - val_accuracy: 0.4914 - val_loss: 1.4492 - learning_rate: 0.0010
Epoch 4/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 161s 269ms/step - accuracy: 0.4575 - loss: 1.5479 - val_accuracy: 0.4767 - val_loss: 1.4761 - learning_rate: 0.0010
Epoch 5/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 162s 270ms/step - accuracy: 0.4742 - loss: 1.5255 - val_accuracy: 0.4991 - val_loss: 1.4323 - learning_rate: 2.0000e-04
Epoch 6/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 164s 273ms/step - accuracy: 0.4706 - loss: 1.5230 - val_accuracy: 0.4971 - val_loss: 1.4340 - learning_rate: 2.0000e-04
Epoch 7/15
598/598 ━━━━━━━━━━━━━━━━━━━━ 162s 271ms/step - accuracy:

In [ ]:
# NEW CELL: Phase 2 - Fine-Tuning Setup

# 1. Unfreeze the base model
base_model.trainable = True

# 2. Freeze all but the last 30 layers 
# (Layers 0 to N-31 are frozen; the top 30 are trainable)
for layer in base_model.layers[:-80]:
    layer.trainable = False

# 3. Recompile the model with a very low learning rate
print("--- Recompiling model for Fine-Tuning ---")
model_transfer_learning.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), # CRITICAL: Very low LR
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 4. Continue training for another 15 epochs 
print("--- Starting Phase 2: Fine-Tuning (Unfrozen Top Layers) ---")
history_2 = model_transfer_learning.fit(
    train_ds,
    validation_data=val_ds,
    epochs=70, # Total target epochs (15 initial + 15 fine-tune)
    initial_epoch=history.epoch[-1] + 1, # <--- CORRECTED: Changed history_1 to history
    callbacks=[lr_scheduler]
)

--- Recompiling model for Fine-Tuning ---
--- Starting Phase 2: Fine-Tuning (Unfrozen Top Layers) ---
Epoch 16/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 275s 363ms/step - accuracy: 0.4422 - loss: 1.5993 - val_accuracy: 0.5311 - val_loss: 1.3659 - learning_rate: 1.0000e-05
Epoch 17/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 162s 270ms/step - accuracy: 0.5256 - loss: 1.3734 - val_accuracy: 0.5700 - val_loss: 1.2584 - learning_rate: 1.0000e-05
Epoch 18/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 162s 270ms/step - accuracy: 0.5716 - loss: 1.2497 - val_accuracy: 0.5908 - val_loss: 1.1913 - learning_rate: 1.0000e-05
Epoch 19/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 162s 269ms/step - accuracy: 0.6031 - loss: 1.1633 - val_accuracy: 0.6269 - val_loss: 1.0985 - learning_rate: 1.0000e-05
Epoch 20/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 163s 271ms/step - accuracy: 0.6204 - loss: 1.1000 - val_accuracy: 0.6486 - val_loss: 1.0418 - learning_rate: 1.0000e-05
Epoch 21/70
598/598 ━━━━━━━━━━━━━━━━━━━━ 165s 274ms/step - accuracy: 0.6498 - loss: 1.0350 - 

In [12]:
## NEW CELL: Final Evaluation

print("--- Running final evaluation on validation set ---")
# Evaluate on the validation dataset (val_ds)
loss, acc = model_transfer_learning.evaluate(val_ds, verbose=1)

print(f"\nFinal Validation Loss: {loss:.4f}")
print(f"Final Validation Accuracy: {acc*100:.2f}%")

--- Running final evaluation on validation set ---
326/326 ━━━━━━━━━━━━━━━━━━━━ 45s 111ms/step - accuracy: 0.5016 - loss: 1.4258

Final Validation Loss: 1.4312
Final Validation Accuracy: 49.40%


In [13]:
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf

# 1. Collect all images and labels into sequential NumPy arrays
X_test = []
Y_true = []

# Iterate over a clean, unbatched version of the validation dataset to ensure order
for image, label in val_ds.unbatch().as_numpy_iterator():
    X_test.append(image)
    Y_true.append(label)

X_test = np.array(X_test)
Y_true = np.array(Y_true)

print(f"Collected {len(X_test)} total validation images and labels.")

# 2. Make predictions on the collected NumPy array
# This ensures the predictions match the order of Y_true
print("--- Generating Predictions on Reconstructed Validation Data ---")
Y_pred_probabilities = model_transfer_learning.predict(X_test)
Y_pred_classes = np.argmax(Y_pred_probabilities, axis=1)

# 3. Print the final, corrected classification report
print("\n--- Corrected Detailed Classification Report (Final Model) ---")
print(classification_report(Y_true, Y_pred_classes, target_names=class_names))

# 4. Final Sanity Check
accuracy_check = np.mean(Y_pred_classes == Y_true) * 100
print(f"\nOverall Accuracy Check (from Report Data): {accuracy_check:.2f}%")

Collected 10407 total validation images and labels.
--- Generating Predictions on Reconstructed Validation Data ---
326/326 ━━━━━━━━━━━━━━━━━━━━ 51s 128ms/step

--- Corrected Detailed Classification Report (Final Model) ---
                          precision    recall  f1-score   support

   bacterial_leaf_blight       0.36      0.23      0.28       479
   bacterial_leaf_streak       0.32      0.40      0.36       380
bacterial_panicle_blight       0.60      0.79      0.68       337
                   blast       0.69      0.41      0.52      1738
              brown_spot       0.26      0.76      0.38       965
              dead_heart       0.91      0.67      0.77      1442
            downy_mildew       0.34      0.46      0.39       620
                   hispa       0.67      0.23      0.34      1594
                  normal       0.54      0.76      0.63      1764
                  tungro       0.59      0.19      0.29      1088

                accuracy                        

In [14]:
# Cell 11: Modified Plotting Code

import matplotlib.pyplot as plt

# Combine metrics from both phases
acc = history_1.history['accuracy'] + history_2.history['accuracy']
val_acc = history_1.history['val_accuracy'] + history_2.history['val_accuracy']
loss = history_1.history['loss'] + history_2.history['loss']
val_loss = history_1.history['val_loss'] + history_2.history['val_loss']

# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Combined Model Accuracy (Feature Extraction + Fine-Tuning)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.axvline(x=15, color='r', linestyle='--', label='Fine-Tuning Starts') # Mark the transition
plt.legend()
plt.show()


# Plot Loss
plt.figure(figsize=(12, 5))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Combined Model Loss (Feature Extraction + Fine-Tuning)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.axvline(x=15, color='r', linestyle='--', label='Fine-Tuning Starts')
plt.legend()
plt.show()

NameError: name 'history_1' is not defined

In [ ]:
import tensorflow as tf
from tensorflow import keras

model_transfer_learning.save('my_trained_model.keras')
print("Model successfully saved to 'my_trained_model.keras'")


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('my_trained_model.keras') # Or 'my_trained_model.h5'

# Verify the model architecture (optional)
loaded_model.summary()

# The model is now ready to be used!


In [ ]:
import numpy as np

np.save("class_names.npy", class_names)

print("Model and class names saved!")

In [ ]:
!pip install gradio

import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
 

model = tf.keras.models.load_model("my_paddy_model.keras")

class_names = np.load("class_names.npy", allow_pickle=True)

def predict_image(pil_image):
   
    img = pil_image.resize((224, 224))
    
    
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    img_batch = np.expand_dims(img_array, axis=0)
    
    processed_image = preprocess_input(img_batch)
    
    
    prediction = model.predict(processed_image)
    
    scores = prediction[0]
    
    confidences = {class_names[i]: float(scores[i]) for i in range(len(class_names))}
    
    return confidences

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Paddy Leaf Image"),
    outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
    title="Paddy Disease Classifier",
    description="Upload an image of a paddy leaf to classify the disease. This model uses an EfficientNetV2S base."
)

iface.launch(share=True)